#### Clean text

In [ ]:
import re

def merge_spaces(input_string):
    output_string = re.sub(r'\s+', ' ', input_string)
    return output_string

##### eLife Dataset Process (to the plos jsonl format)

In [ ]:
import json
file_path = './datasets/elife/test.json'
with open(file_path, 'r') as file:
    json_objs = json.load(file)

In [ ]:
import jsonlines

edatas = []

for json_obj in json_objs:

    edata = {}

    abstract = ' '.join(json_obj['abstract']).replace('\n',' ')
    lay = ' '.join(json_obj['summary']).replace('\n',' ')

    assert len(json_obj["sections"]) == len(json_obj['headings'])
    cur_articles = []
    for nid, (article_list, heading) in enumerate(zip(json_obj["sections"],json_obj['headings'])):

        cur_articles.append(str(nid+1) + ' ' + heading + ' ' + ' '.join(article_list).replace('\n',' '))

    edata['doi'] = json_obj['id']
    edata['abstract'] = abstract
    edata['plain language summary'] = lay
    edata['article'] = ' '.join(cur_articles)

    edatas.append(edata)


with jsonlines.open('./datasets/elife/test_elife.jsonl', 'w') as writer:
    for edata in edatas:
        writer.write(edata)

#### extract instance's article (both datasets)

In [ ]:
import json
# import textstat

plos_datas_ts = []
with open('./datasets/plos/test_plos.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        plos_datas_ts.append(data)


elife_datas_ts = []
with open('./datasets/elife/test_elife.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        elife_datas_ts.append(data)

In [ ]:
with open('./datasets/elife/elife_testset.txt', 'a+', encoding="utf-8") as f:
    for data in elife_datas_ts:
        f.write(data['article'].strip() + '\n')

with open('./datasets/plos/plos_testset.txt', 'a+', encoding="utf-8") as f:
    for data in plos_datas_ts:
        f.write(merge_spaces(data['article'].replace('\n',' ')).strip() + '\n')

#### Calculate each QA in predicted QA pairs score with the reference of golden summaries (for ablation study)

In [ ]:
import json
# import textstat

gdatas = []
with open('./datasets/plos/test_plos.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        data = json.loads(lines[lid])
        gdatas.append(data)

In [ ]:
from rouge_score import rouge_scorer
import numpy as np

def calc_rouge(preds, refs):
  # Get ROUGE F1 scores
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], \
                                    use_stemmer=True, split_summaries=True)
  scores = [scorer.score(p, refs[i]) for i, p in enumerate(preds)]
  # print([s['rougeL'].fmeasure for s in scores])
  # return np.mean([s['rouge1'].fmeasure for s in scores]), \
  #        np.mean([s['rouge2'].fmeasure for s in scores]), \
  #        np.mean([s['rougeLsum'].fmeasure for s in scores])
  return [str(s['rougeLsum'].fmeasure) for s in scores]

def read_file_lines(path):
  with open(path, 'r') as f:
    lines = f.readlines()
  if path.endswith('.jsonl'):
    lines = [json.loads(line) for line in lines]
  return lines

def evaluate(preds, refs):
  return calc_rouge(preds, refs)

In [ ]:
import re
with open('./qa_pairs/predict/plos/pls_qa.txt', 'r', encoding="utf-8") as f:
    # lines = f.readlines()[:100]
    for lid, line in enumerate(f.readlines()[:100]):
        line = re.sub(r'\b\d+:', '\nQuestion:', line.split('\t')[-1]).replace('Answer:'," Answer:").strip()
        preds_ = [pred.replace("Question: ","").replace("Answer: ","") for pred in line.split("\n")]
        assert len(preds_) == 10
        # refs_ = [merge_spaces(gdatas[lid]['plain language summary'].replace('\n',' ')).strip()] * 10
        refs_ = [merge_spaces(gdatas[lid]['abstract'].replace('\n',' ')).strip()] * 10
        scores = evaluate(preds_, refs_)
        with open("./qa_pairs/predict/plos/pls_qa_scores.txt","a+") as f:
            f.write("\t".join(scores) + "\n")

#### involking LLM API

In [ ]:
## for summary generation

import time
import openai
import json
from tqdm import tqdm
openai.api_type = "azure"
openai.api_version =  
openai.api_base = "https://gpt4apidutir.openai.azure.com/"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = 

# t_sum = 0

with open('./datasets/plos/test_elife.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    for lid in tqdm(range(241)):
        data = json.loads(lines[lid])
        # context = data['article'].replace('\n',' ').strip()
        context = ' '.join(data['article'].replace('\n',' ').strip().split(" "))
        # print(len(context.split(" ")))

        conversation=[
            {"role": "user", "content": "Below is a paper. \n {} \n Please give me a layman summary start with [L] and an expert summary start with [E] of this paper in one paragraph seperately.".format(context)},
            ]
                
        response = openai.ChatCompletion.create(
            engine="", # The deployment name you chose when you deployed the GPT-35-16k or GPT-4-turbo model.
            messages=conversation
        )


        with open('', 'a+', encoding='utf-8') as f:
            f.write(data['doi'] + '\t' + response['choices'][0]['message']['content'].strip().replace('\n','') + '\n')
               
        time.sleep(15) 

In [ ]:
## for qa pairs generation

datas = []
with open('./datasets/plos/test_plos.jsonl', 'r', encoding="utf-8") as f:
    lines = f.readlines()
    # print(len(lines)) 26214
    for lid in range(len(lines)):
        datas.append(json.loads(lines[lid]))

for nid, data in enumerate(tqdm(datas)):
    
    for id in range(2):

        # summary = data[id]
        # conversation=[
        # {"role": "user", "content": "Generate five answerable and specific questions based on the following context. Context: {} \n and give me the answers of them".format(summary)},
        # ]
        if id % 2 == 1:
            summary = merge_spaces(data["plain language summary"].replace('\n',' ')).strip()
        else:
            summary = merge_spaces(data["abstract"].replace('\n',' ')).strip()

        conversation=[
        {"role": "user", "content": "Text: {} \n Given the above text, please propose 10 English questions that are diverse and cover all parts of the text, in the following format: \"1: \", \"2: \", ..., and give me the answer of each question.".format(summary)},
        ]


        response = openai.ChatCompletion.create(
            engine="",
            messages=conversation
        )


        if id % 2 == 1:
            with open('', 'a+', encoding='utf-8') as f:
                f.write(data['doi'] + '\t' + response['choices'][0]['message']['content'].replace('\n','') + '\n')
        else:
            with open('', 'a+', encoding='utf-8') as f:
                f.write(data['doi'] + '\t' + response['choices'][0]['message']['content'].replace('\n','') + '\n')
      
    if nid % 3 == 0 and nid != 0:
        time.sleep(10)